In [1]:
import pandas as pd
import numpy as np
np.random.seed(1119)  # for reproducibility
np.set_printoptions(threshold=np.nan)

from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import scipy as sp
import pandas as pd
import sys, os, math, argparse

In [2]:
from keras.models import Model, Sequential
from keras.layers import GRU, LSTM
from keras.layers import Input, Dropout, Dense, Reshape, Flatten, Activation
from keras.layers.merge import _Merge
from keras.layers import Conv1D, Conv2D
from keras.layers.convolutional import Convolution2D, Conv2DTranspose, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation, ZeroPadding2D
from keras.layers import TimeDistributed, RepeatVector
from keras.layers.advanced_activations import LeakyReLU
from keras import optimizers
from keras import losses
from keras.datasets import mnist
from keras.utils import np_utils
from keras.preprocessing import sequence as ksq
from keras import backend as K
from functools import partial
from keras.optimizers import Adam

/home/zhengma/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Import Datasets
df_seq = pd.read_csv('./data/pdb_data_seq.csv')
df_char = pd.read_csv('./data/pdb_data_no_dups.csv')

In [4]:
# Filter for only proteins
protein_char = df_char[df_char.macromoleculeType == 'Protein']
protein_seq = df_seq[df_seq.macromoleculeType == 'Protein']

# Select only necessary variables to join
protein_char = protein_char[['structureId','classification']]
protein_seq = protein_seq[['structureId','sequence']]

In [5]:
model_f=protein_seq.merge(protein_char,on=['structureId']).reset_index(drop=True)

In [6]:
model_f.shape

(346325, 3)

In [7]:
model_f = model_f.dropna()

In [8]:
model_f.shape

(346321, 3)

In [9]:
counts = model_f.classification.value_counts()
# Get classification types where counts are over 1000
types = np.asarray(counts[(counts > 1000)].index)

# Filter dataset's records for classification types > 1000
data = model_f[model_f.classification.isin(types)]

In [10]:
num_classes=len(data['classification'].unique())

In [11]:
num_classes

43

## Here:
we do not care the class lables. As the starting point, we firstly try to generate sequences that do not collapse and make biochemical sense

In [12]:
data.head()

,structureId,sequence,classification
16,10GS,PPYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKAS...,TRANSFERASE/TRANSFERASE INHIBITOR
17,10GS,PPYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKAS...,TRANSFERASE/TRANSFERASE INHIBITOR
27,117E,TYTTRQIGAKNTLEYKVYIEKDGKPVSAFHDIPLYADKENNIFNMV...,HYDROLASE
28,117E,TYTTRQIGAKNTLEYKVYIEKDGKPVSAFHDIPLYADKENNIFNMV...,HYDROLASE
31,11AS,MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...,LIGASE


In [13]:
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer

In [14]:
# maximum length of sequence, everything afterwards is discarded!
max_length = 1024

#create and fit tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(data['sequence'])
#represent input data as word rank number sequences
train_X = tokenizer.texts_to_sequences(data['sequence'])
train_X = pd.DataFrame(sequence.pad_sequences(train_X, maxlen=max_length))

In [15]:
train_X.head()

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0,0,0,0,0,0,0,0,0,0,...,13,1,12,9,13,3,13,3,10,15
1,0,0,0,0,0,0,0,0,0,0,...,13,1,12,9,13,3,13,3,10,15
2,0,0,0,0,0,0,0,0,0,0,...,7,10,19,14,14,9,6,3,6,4
3,0,0,0,0,0,0,0,0,0,0,...,7,10,19,14,14,9,6,3,6,4
4,0,0,0,0,0,0,0,0,0,0,...,2,4,11,5,6,4,12,6,1,1


In [16]:
max(train_X.max())

25

## GAN building

In [80]:
class proteinGAN():
    def __init__(self, n_aa=max_length, n_render=1, latent_dim=10):
        self.n_aa = n_aa
        self.n_render = n_render
        self.pro_shape = (self.n_aa, self.n_render)
        self.latent_dim = latent_dim

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        # Build the generator
        self.generator = self.build_generator()
        # Generated peptide
        z = Input(shape=(self.latent_dim,))
        peptide = self.generator(z)
        # The discriminator takes generated peptides as input and determines validity
        validity = self.discriminator(peptide)
        # For the combined model we will only train the generator
        for layer in self.generator.layers:
            layer.trainable=True
        for layer in self.discriminator.layers:
            layer.trainable=False
        self.generator.trainable=True
        self.discriminator.trainable = False
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        self.combined.summary()
        for layer in self.generator.layers:
            layer.trainable=False
        for layer in self.discriminator.layers:
            layer.trainable=True
        self.generator.trainable=False
        self.discriminator.trainable = True
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])
    
    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_shape=(self.latent_dim,)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.pro_shape), activation='tanh'))
        model.add(Reshape(self.pro_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        peptide = model(noise)

        return Model(noise, peptide)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.pro_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        peptide = Input(shape=self.pro_shape)
        validity = model(peptide)
        return Model(peptide, validity)
    
    def train(self, X, epochs, batch_size=128, sample_interval=50):
        # Load the dataset
        X_train = X

        # Rescale
        X_train = X_train / 25.0

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            peptides = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim ))

            # Generate a new batch
            gen_peptides = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(peptides, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_peptides, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim ))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval
            if epoch % sample_interval == 0:
                self.sample_peptides(epoch)
            
        return self.combined
    
    def sample_peptides(self, epoch):
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        gen_peptides = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_peptides = 25.0 * gen_peptides
        return gen_peptides

In [81]:
np.random.normal(0,1,5).shape

(5,)

In [82]:
pro_gan = proteinGAN()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_13 (Flatten)         (None, 1024)              0         
_________________________________________________________________
dense_85 (Dense)             (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_61 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_86 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_62 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_87 (Dense)             (None, 1)                 257       
Total params: 656,385
Trainable params: 656,385
Non-trainable params: 0
_________________________________________________________________
____

In [83]:
train_X.values.shape

(278866, 1024)

In [84]:
train_X_3 = np.reshape(train_X.values, (train_X.shape[0], train_X.shape[1], 1))

In [85]:
pro_gan.train(X=train_X_3,  epochs=300, batch_size=32, sample_interval=200)

/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.685829, acc.: 39.06%] [G loss: 0.563598]
1 [D loss: 0.575864, acc.: 90.62%] [G loss: 0.474706]
2 [D loss: 0.534127, acc.: 100.00%] [G loss: 0.368378]


/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


3 [D loss: 0.435287, acc.: 100.00%] [G loss: 0.308285]
4 [D loss: 0.390991, acc.: 100.00%] [G loss: 0.213434]
5 [D loss: 0.368589, acc.: 100.00%] [G loss: 0.181872]


/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


...

/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


294 [D loss: 0.015424, acc.: 98.44%] [G loss: 0.001767]
295 [D loss: 0.018184, acc.: 98.44%] [G loss: 0.003209]
296 [D loss: 0.023769, acc.: 98.44%] [G loss: 0.010957]


/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


297 [D loss: 0.021093, acc.: 98.44%] [G loss: 0.008167]
298 [D loss: 0.005130, acc.: 100.00%] [G loss: 0.001472]
299 [D loss: 0.008961, acc.: 100.00%] [G loss: 0.014670]


/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [94]:
tmp_arr = list(np.abs(np.reshape(pro_gan.sample_peptides(1), (max_length,)).astype(np.int)))

In [100]:
alpha=list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

In [104]:
seq = ''.join([alpha[id] for id in tmp_arr])

In [105]:
seq

'CAHBHBKCICAKMNKHEEDOCNRDEFEHOBOMENEIGHGGDBCCNFCNLKFNMBECAKEDQHGPELAKLECADNLCDBENCNCOOGOFGEOAFFLCPBLDJMFPIJLHCMFAAADEAKDGCMFPJBBFEEEFFDCNQAFAGCJBIHDGQFAAIGFJDMGEHCCDJFFJIDHCBHCLGAKKFNIIBLFAFGHMIHLEIJCNKANHHBKCIAOFLODIHMDKDCJKRCELRCAPBLKFAQKDBBBQCFENDTJRHDHOGAMCGIEECDKDGAACKGFHDCEIEGBEPCEHCEECADJFMBFKMEFPJFEINAELMLKEBBCHIQLDAJHKDFCJCDKFLEALDFHMSFMGFBCPKDFFGEGKMIFCAJFBOEMFFABDGFMJCOCBEFLECFKAKLDDJNHDLBMFDMMCBHKNHGELCFCAGIDABNBABBDPAGCDIDEDINKEINBEKDEFCNBFAHHMLGNAJAAFUCGNSIEMHKOQQANLDLQDAECIFFAGFDUIAPEEADIJMJLCHGEBJPGIOGAOKBDBOJPCLJFFPBEAMIGAGGGRMGMGHHKMIGMKGBOHGMFIDCAGFLHIFKTDCCBAAHPADHDIEIJCBCFFEFAHGGJABGDNJADFLNIDADKDBBQELJDIKDJJNCBVDJCMDDAEOEHBBFHEFEFEAAHCEEHHHBNGADCEACACEDACMCEIMKIDHJAFJJJMEEOHJAAJAEKGEASFCDAPBHJDDNLDEDDCLACHAUBKNBIQDGEDFBFBMDCFAEEEBEDJAILGHGFPGBBMKAOCFHCFOMLFQNBPGUDDQBOFALINEBJKIBCCPFCEEKPJBEIRFLCEBBODBEEBEIAEDUCKFGFAFCFSSPUEBCMBUJFEORCQLEGPCIQUCEBBKCCBAIQFJAJQBTMLJSKQBQAHRBCABUCAACJMCAEKGBFQMGHICBEEABMBCDKHJBEEMJMBEBINODFDBJABCAALJCHBHEDEGENBIBQSHDIIGEFJDOGKBOJBANF

## xtmp

In [25]:
class xtmp():
    def __init__(self, arg1):
        self.arg1 = arg1
    def print_arg1(self):
        print(self.arg1)
    

In [27]:
xtmp1 = xtmp('KK')

In [28]:
xtmp1.print_arg1()

KK


In [69]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        self.combined.summary()


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        # fig.savefig("images/%d.png" % epoch)
        plt.close()

In [70]:
gan = GAN()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_11 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_71 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_51 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_72 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_52 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_73 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

In [71]:
gan.train(epochs=30000, batch_size=32, sample_interval=200)

/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.913857, acc.: 40.62%] [G loss: 0.787012]


/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning:

1 [D loss: 0.430321, acc.: 75.00%] [G loss: 0.825831]
2 [D loss: 0.269279, acc.: 92.19%] [G loss: 0.944387]
3 [D loss: 0.263517, acc.: 92.19%] [G loss: 1.152484]
4 [D loss: 0.223368, acc.: 93.75%] [G loss: 1.266998]


/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning:

...

889 [D loss: 0.711484, acc.: 46.88%] [G loss: 0.785870]
890 [D loss: 0.639935, acc.: 57.81%] [G loss: 0.795861]
891 [D loss: 0.627795, acc.: 64.06%] [G loss: 0.824698]
892 [D loss: 0.618306, acc.: 70.31%] [G loss: 0.879817]


/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning:

893 [D loss: 0.635592, acc.: 56.25%] [G loss: 0.866075]
894 [D loss: 0.649227, acc.: 59.38%] [G loss: 0.817096]
895 [D loss: 0.632775, acc.: 62.50%] [G loss: 0.840715]
896 [D loss: 0.616492, acc.: 65.62%] [G loss: 0.838816]


/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/zhengma/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning:

KeyboardInterrupt: 